# 필요한 라이브러리 import

In [ ]:
#import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# Stanford Dogs 데이터셋 다운로드 및 압축 해제

In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!tar -xf images.tar

--2024-02-06 07:11:10--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  72.2MB/s    in 11s     

2024-02-06 07:11:21 (68.7 MB/s) - ‘images.tar’ saved [793579520/793579520]



# 데이터 준비

- Stanford Dogs 데이터셋을 학습 및 검증 데이터셋으로 나누어 줍니다.
- Keras의 ImageDataGenerator를 사용하여 데이터를 로드하고 전처리합니다.


In [ ]:
#훈련 데이터 증강

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest",
                                   validation_split=0.2
                                   )

In [ ]:
#훈련 데이터 생성
train_generator = train_datagen.flow_from_directory(
    "Images", target_size=(224,224), batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 16508 images belonging to 120 classes.


In [ ]:
#검증 데이터 생성
validation_generator = train_datagen.flow_from_directory(
    "Images", target_size=(224,224), batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 4072 images belonging to 120 classes.


# 모델구성

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
# 기본 모델의 파라미터를 동결
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
# 커스텀 레이어 추가 - 분류기 모델
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)

# 최종 레이어
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

In [ ]:
# 새로운 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss="categorical_crossentropy", metrics=["acc"])

# 모델 학습

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)

Epoch 1/10
515/515 [==============================] - 233s 438ms/step - loss: 1.5334 - acc: 0.5887
Epoch 2/10
515/515 [==============================] - 226s 439ms/step - loss: 0.8634 - acc: 0.7380
Epoch 3/10
515/515 [==============================] - 223s 433ms/step - loss: 0.7279 - acc: 0.7695
Epoch 4/10
515/515 [==============================] - 222s 432ms/step - loss: 0.6468 - acc: 0.7958
Epoch 5/10
515/515 [==============================] - 225s 437ms/step - loss: 0.5759 - acc: 0.8146
Epoch 6/10
515/515 [==============================] - 225s 436ms/step - loss: 0.5277 - acc: 0.8256
Epoch 7/10
515/515 [==============================] - 222s 431ms/step - loss: 0.4811 - acc: 0.8423
Epoch 8/10
515/515 [==============================] - 222s 430ms/step - loss: 0.4381 - acc: 0.8560
Epoch 9/10
515/515 [==============================] - 224s 436ms/step - loss: 0.4064 - acc: 0.8656
Epoch 10/10
515/515 [==============================] - 223s 433ms/step - loss: 0.3803 - acc: 0.8715


# 모델 평가

In [ ]:
# 모델 평가
eval_result = model.evaluate(validation_generator)

print(f'Validation loss: {eval_result[0]}, Validation accuracy: {eval_result[1]}')

128/128 [==============================] - 57s 434ms/step - loss: 1.2651 - acc: 0.7053
Validation loss: 1.2650537490844727, Validation accuracy: 0.7053045034408569
